## Optimization Problem

Let the given graph(network) is represented by $G=(V,E)$. For each vertex $u \in V(G)$, we associate a hyper-sphere of radius $R_u$ and center at $C_u$. Let us denote the set of radius as $R=\{R_u: u \in V(G)\}$ and $C = \{C_u: u \in V(G)\}$. If the hyper-sphere is $K$-dimensional, then $R \in {\rm I\!R}^{|V|}$ and $C \in {\rm I\!R}^{|V| \times K}$.

Let $C_u$ and $R_u$ and $C_v$ and $R_v$ be the center and radius of two $K$-dimensional hyper-spheres corresponding to vertices $u, v$ respectively. $C_u, C_v \in {\rm I\!R}^K$ and $R_u, R_v \in {\rm I\!R}$. Let $X_{uv}$ be the embedding of the edge $e=(u,v) \in E(G)$. Let us denote the embedding set as $X=\{X_{uv}: (u,v) \in E\}$.

The optimization problem we are trying to solve here is as follows:
<br>

Minimize &nbsp; $\alpha \sum_{u \in V}{R_u^2} + \sum_{v \in V_L}\big[|N_s(v)|\log \sum_{\bar{v}\in N_s(v)}\exp(X_{\bar{v}}.X_v) - \sum_{v' \in N_s(v)}{X_{v'}.X_v}\big]$ <br>
subject to $\left\Vert X_{uv} - C_{u} \right\Vert^2 \leq R_u^2$  and  $\left\Vert X_{uv} - C_{v} \right\Vert^2 \leq R_v^2 \quad \forall (u,v) \in E$
            
The optimization variables in this problem are $X, R, C$.

### Solve the optimization using Penalty Method

The above optimization problem can be reformulated, using the method of adding a penalty term, as follows:

Minimize &nbsp; $\sum_{v \in V_L}\big[|N_s(v)|\log \sum_{\bar{v}\in N_s(v)}\exp(X_{\bar{v}}.X_v) - \sum_{v' \in N_s(v)}{X_{v'}.X_v}\big] + \alpha \sum_{u \in V}{R_u^2} + \beta \sum_{u \in V} \sum_{v \in N(u)} g\big( \left\Vert X_{uv} - C_{u} \right\Vert^2 - R_u^2 \big)$ <br>

Here, we have used RELU function for $g$, defined as:

$g(t) = t * \mathbb{1}(t>0)$, where $\mathbb{1}$ is the standard indicator function.

We try to solve the previous optimization problem using $\textbf{Alternating Gradient Update}$ method. The first step is to solve the third summation term in the above objective function using node2vec method. Then we update $X_{uv}$ by subtracting the gradient of the second term. This we do keeping $R$ and $C$ fixed. Next step is to update $R$ and $C$, by subtracting the gradient of the first and third term w.r.t. $R$,$C$ and keeping $X_{uv}$ constant. The gradient calculation is mentioned below. 

In [1]:
from scipy.optimize import minimize
import numpy as np

## Updation of embedding on using penalty method

Once we have the embeddings $X_{uv}, \forall (u,v) \in E(G)$ obtained from node2vec method, we update the embeddings by calculating the gradient of the third term in the optimization objective function. The entire update process is as follows:

$\nabla_{X_{uv}} F_3 = 2 \beta (X_{uv} - C_u) * \mathbb{1}\big( \left\Vert X_{uv} - C_{u} \right\Vert^2 - R_u^2 \big) + 2 \beta (X_{uv} - C_v) * \mathbb{1}\big( \left\Vert X_{uv} - C_{v} \right\Vert^2 - R_v^2 \big)$

$X_{uv} = X_{uv} - \eta * \nabla_{X_{uv}} F_3$

Here, $F_3$ is the third term in the optimization problem.



In [50]:
def update_embeddings(embeddings, centers, radii, edge_map, nodes, beta=0.001, eta=0.1):
    edge_count = embeddings.shape[0]
    embed_dim = embeddings.shape[1]
    assert edge_count == len(edge_map.keys())
    for i in range(edge_count):
        edge = edge_map[i]
        n_u = edge[0]
        n_v = edge[1]
        X_uv = embeddings[i]
        n_u_ind = np.where(nodes == n_u)
        n_v_ind = np.where(nodes == n_v)
        c_u = centers[n_u_ind]
        c_v = centers[n_v_ind]
        r_u = radii[n_u_ind]
        r_v = radii[n_v_ind]
        dX_uv = np.zeros((1, embed_dim))
        if np.linalg.norm(X_uv - c_u) > r_u:
            dX_uv += 2 * beta * (X_uv - c_u)
        if np.linalg.norm(X_uv - c_v) > r_v:
            dX_uv += 2 * beta * (X_uv - c_v)
        embeddings[i] = X_uv - eta * dX_uv
    
    return embeddings

### Test for the update of embeddings in the above optimization problem

In [51]:
embeddings = np.array([[2, 0], [6, 0], [10, 0]], dtype=float)
centers = np.array([[0, 0], [4, 0], [8, 0], [12, 0]], dtype=float)
radii = np.array([[1], [3], [3], [1]], dtype=float)
edge_map = {0:(1, 2), 1:(2, 3), 2:(3, 4)}
nodes = np.array([1, 2, 3, 4])
new_embeddings = update_embeddings(embeddings, centers, radii, edge_map, nodes)
print(new_embeddings)

[[  1.9996   0.    ]
 [  6.       0.    ]
 [ 10.0004   0.    ]]


## Updation of Sphere : radius length and center locations

The third and final step of the alternating gradient update technique we have used is to calculate the gradient combining second and third terms in the optimization objective and then update $X$ and $C$, keeping $X_{uv}$ constant. Let us calculate the gradients and modify the radii and centers, mathematically:

$\begin{equation}
\frac{\partial F}{\partial R_u} = 2 \alpha R_u - 2 \beta R_u \sum_{v \in N(u)}{\mathbb{1}\big( \left\Vert X_{uv} - C_{v} \right\Vert^2 - R_v^2 \big)}
\end{equation}$

$R_u = R_u - \eta * \frac{\partial F}{\partial R_u}$

$\nabla_{C_u} F= 2 \beta \sum_{v \in N(u)}{\big(C_u - X_{uv}\big) * \mathbb{1}\big( \left\Vert X_{uv} - C_{v} \right\Vert^2 - R_v^2 \big) }$

$C_u = C_u - \eta * \nabla_{C_u} F$

In the above, $F = F_2 + F_3$, where $F_2$, $F_3$ represent the second and third term respectively in the objective function.

In [55]:
def update_sphere(embeddings, centers, radii, edge_map, nodes, alpha=0.3, beta=0.1, eta=0.1):
    # Update radius and centers using gradients
    node_count = len(nodes)
    edge_count = embeddings.shape[0]
    embed_dim = embeddings.shape[1]
    dradii = np.zeros((node_count, 1))
    dcenters = np.zeros((node_count, embed_dim))
    for i in range(edge_count):
        edge = edge_map[i]
        n_u = edge[0]
        n_v = edge[1]
        n_u_ind = np.where(nodes == n_u)
        n_v_ind = np.where(nodes == n_v)
        X_uv = embeddings[i]
        c_u = centers[n_u_ind]
        c_v = centers[n_v_ind]
        r_u = radii[n_u_ind]
        r_v = radii[n_v_ind]
        if np.linalg.norm(X_uv - c_u) > r_u:
            dradii[n_u_ind] -= 2*beta*r_u
            dcenters[n_u_ind] += 2*beta*(c_u - X_uv)
        
        if np.linalg.norm(X_uv - c_v) > r_v:
            dradii[n_v_ind] -= 2*beta*r_v
            dcenters[n_v_ind] += 2*beta*(c_v - X_uv)
    
    for i in range(node_count):
        r_u = radii[i]
        dradii[i] += 2*alpha*r_u
        
    radii -= eta * dradii
    centers -= eta * dcenters

    return radii, centers

### Test for update of radii and centers in the above optimization problem

In [56]:
embeddings = np.array([[2, 0], [6, 0], [10, 0]], dtype=float)
centers = np.array([[0, 0], [4, 0], [8, 0], [12, 0]], dtype=float)
radii = np.array([[1.], [2.], [2.], [1.]], dtype=float)
edge_map = {0:(1, 2), 1:(2, 3), 2:(3, 4)}
nodes = np.array([1, 2, 3, 4])
updated_radii, updated_centers = update_sphere(embeddings, centers, radii, edge_map, nodes)
print(updated_centers)
print(updated_radii)

[[  0.04   0.  ]
 [  4.     0.  ]
 [  8.     0.  ]
 [ 11.96   0.  ]]
[[ 0.96]
 [ 1.88]
 [ 1.88]
 [ 0.96]]
